In [ ]:
# Installations
!pip install langchain | tail -n 1
!pip install langchain openai transformers chromadb | tail -n 1
!pip install pypdf | tail -n 1
!pip install sentence-transformers | tail -n 1
!pip install "langchain-chroma>=0.1.2" | tail -n 1
!pip install -U langchain-community | tail -n 1
!pip install openai transformers chromadb pypdf sentence-transformers "langchain-chroma>=0.1.2" -U langchain-community | tail -n 1

In [ ]:
# Imports
import os
import time
from google.colab import userdata
import langchain as lc
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain_core.exceptions import OutputParserException
from langchain.chat_models import ChatOpenAI

In [ ]:
# Set up OpenAI API Key
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

In [ ]:
# Load and Process Documents
document_paths = [
    "/content/indiana15_ceds_2023.pdf",
    "/content/eda_ceds_guidelines_2023.pdf"
]

documents = []
for path in document_paths:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())

In [ ]:
# Improved Text Chunking and Vectorization
# Increased chunk size for better context
text_splitter = lc.text_splitter.RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-distilroberta-v1")
vector_store = Chroma.from_documents(chunks, embedding_model)


<ipython-input-4-cd3df89ba4d6>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-distilroberta-v1")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

In [ ]:
# Initialize OpenAI LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")

<ipython-input-5-9376f39a8862>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo")


In [ ]:
# Set Up Retrieval-QA Chain for RAG
retriever = vector_store.as_retriever()

def dynamic_document_retrieval(query, context_doc=None):
    global retriever

    if context_doc:
        filtered_chunks = [chunk for chunk in chunks if context_doc.lower() in chunk.page_content.lower()]
        if not filtered_chunks:
            return {"output": "Context document not found."}

        filtered_vector_store = Chroma.from_documents(filtered_chunks, embedding_model)
        retriever = filtered_vector_store.as_retriever()
    else:
        retriever = vector_store.as_retriever()

    rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)
    return rag_chain.invoke({"query": query})

retrieval_tool = Tool(
    name="document_retriever",
    func=dynamic_document_retrieval,
    description="Retrieves information from selected documents on demand"
)

In [ ]:
# Improved Agent Prompt for SWOT Analysis with Document-Specific Clues
agent_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],
    template="""
    You are an economic development analysis assistant specializing in SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis.
    You have access to multiple documents in the CEDS database. Extract relevant information and format your output accordingly.

    Look for keywords such as "advantage," "strength," "challenge," "opportunity," "risk," or "threat" to identify each section of the SWOT analysis.

    Follow this format:
    - **Strengths**: Provide specific strengths of the region with examples.
    - **Weaknesses**: Provide specific weaknesses of the region with examples.
    - **Opportunities**: Provide specific opportunities for the region with examples.
    - **Threats**: Provide specific threats to the region with examples.

    Provide each section as a separate list item with clear, concise points.

    If a section is missing, respond with "No relevant information found" for that part.

    Thought: {agent_scratchpad}
    Query: {input}
    """
)

In [ ]:
# Initialize the Agent
tools = [retrieval_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    prompt=agent_prompt,
    handle_parsing_errors=True
)

# Define Agentic Query Function with Retry Mechanism
# Added fallback query to improve analysis

def agentic_query(query, context_doc=None, retries=3):
    input_data = {
        "input": query,
        "agent_scratchpad": "",
    }
    for attempt in range(retries):
        try:
            time.sleep(1)  # Delay to respect API rate limit
            response = agent.invoke(input_data)
            if isinstance(response, dict):
                output = response.get('output', 'No relevant information found.')
                if "No relevant information found" in output:
                    print(f"Attempt {attempt + 1} - No relevant information found. Retrying with adjusted query...")
                    input_data["input"] = "Reattempting to extract SWOT information with a focus on specific keywords like advantage, challenge, and risk."
                else:
                    return output
            else:
                return response
        except OutputParserException as e:
            print(f"Attempt {attempt + 1} - Error processing query: {str(e)}")
        except Exception as e:
            print(f"Attempt {attempt + 1} - Unexpected error: {str(e)}")
    return "Exceeded maximum retry attempts without successful completion."

<ipython-input-8-24372b2de7ef>:4: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [ ]:
result = agentic_query("Provide a detailed SWOT analysis based on the content in the Indiana 15 CEDS document.")



> Entering new AgentExecutor chain...
I need to retrieve the Indiana 15 CEDS document to analyze its content for a SWOT analysis.
Action: document_retriever
Action Input: "Indiana 15 CEDS"
Observation: {'query': 'Indiana 15 CEDS', 'result': "The Indiana 15 CEDS stands for the Comprehensive Economic Development Strategy for the Indiana 15 region. It is a strategic plan that outlines the region's economic development goals and priorities for the next five years. The CEDS is informed by the region's vision statement, which includes aspirations for economic growth, tourism, quality of life, and opportunities for residents. The plan focuses on areas like population growth, housing, entrepreneurship, business development, education, income levels, and community revenues. The Indiana 15 RPC (Regional Planning Commission) plays a key role in coordinating and implementing this strategy.", 'source_documents': [Document(metadata={'page': 3, 'source': '/content/indiana15_ceds_2023.pdf'}, page_co